<a href="https://colab.research.google.com/github/michaeledge27/mathModeling/blob/main/projects/permanentProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def permRecurse(row, used, mat):
    N = len(mat[0])
    v = 0
    if row == N:
        return 1
    else:
      for j in range(N):
        if (not used[j] and mat[row][j] != 0):
          used[j] = True
          v += permRecurse(row + 1, used, mat)
          used[j] = False
      return v

In [3]:
# return the number of ones in a given matrix
def onesCount(matrix):
    return np.count_nonzero(matrix)

In [4]:
def generate_matrix(n: int, m: int):
    # generate nxn matrix with all 0's
    matrix = np.zeros((n, n), dtype=int)
    options = n**2
    # randomly select m unique indices from the n**2 options
    indices = np.random.choice(options, m, replace=False)
    # replace the selected elements of the 0's matrix with 1's
    np.put(matrix, indices, 1)
    return matrix

In [5]:
def chromosones(n: int, m: int, population: int):
    # create a list of chromosones (nxn 0/1 matrices having m 1's)
    total_population = []
    # generate the population size # of chromosones
    for i in range(population):
        # generate nxn matrix with m 1's
        matrix = generate_matrix(n, m)
        total_population.append(matrix)
    # return population
    return total_population

In [6]:
population = chromosones(8, 20, 3)
population[0]


array([[0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 1, 1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0]])

In [7]:
population[0][1]

array([0, 1, 0, 1, 1, 0, 1, 0])

In [8]:
def fitness_function(matrix):
    return permRecurse(row=0, used=[False] * len(matrix[0]), mat=matrix)

In [9]:
fitness_function(population[0])

7

In [10]:
def pick_direction(i, j):
    direction = np.random.choice(['north', 'south', 'east', 'west'])
    if direction == 'north':
        i -= 1
    elif direction == 'south':
        i += 1
    elif direction == 'east':
        j += 1
    elif direction == 'west':
        j -= 1
    return i, j

In [11]:
pick_direction(1, 1)

(1, 2)

In [12]:
def select_element(matrix):
    i = np.random.randint(0, len(matrix))
    j = np.random.randint(0, len(matrix[0]))
    return i, j

In [13]:
select_element(population[0])

(2, 3)

In [14]:
def check_same(element, new_element):
    return element == new_element

In [15]:
check_same(2, 2)

True

In [16]:
check_same(2, 4)

False

In [17]:
# return false if at least one of the NSEW neighbors is different
def check_all_neighbors(matrix, i, j):
    return matrix[i][j] == matrix[i-1][j] and matrix[i][j] == matrix[i+1][j] and matrix[i][j] == matrix[i][j-1] and matrix[i][j] == matrix[i][j+1]


In [18]:
check_all_neighbors(population[0], 0, 0)

False

In [19]:
check_all_neighbors(population[0], 5, 2)

False

In [38]:
def perform_swap(matrixA, i, j, new_i, new_j):
    matrixA[i][j], matrixA[new_i][new_j] = matrixA[new_i][new_j], matrixA[i][j]
    return matrixA


In [39]:
def mutation(matrixA, matrixB):
    i, j = select_element(matrix)
    if check_all_neighbors(matrix, i, j):
        return matrix
    else:
        new_i, new_j = pick_direction(i, j)
        if check_same(i, new_i) and check_same(j, new_j):
            return matrix
        else:
            return perform_swap(matrix, i, j, new_i, new_j)


In [23]:
mat = [[1, 1, 0],
       [1, 1, 0],
       [0, 0, 1]]
N = len(mat[0])
used = [False] * N
v = permRecurse(0, used, mat)
print(v)

2


In [26]:
matrix = generate_matrix(8, 20)
matrix

array([[0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0]])

In [27]:
N = len(matrix[0])
used = [False] * N
v = permRecurse(0, used, matrix)
print(v)

3


In [33]:
def calculate_max_perm(n, m, trials):
  permanents = []
  for i in range(trials):
    matrix = generate_matrix(n, m)
    N = len(matrix[0])
    used = [False] * N
    v = permRecurse(0, used, matrix)
    permanents.append(v)
  return max(permanents)


In [37]:
calculate_max_perm(8, 20, 1000)

16